In [26]:
import pandas as pd
import csv
import datetime
from dateutil.parser import parse
import os
!pwd

/Users/sbhasin/Desktop/PandemicChallenge/timeseries


In [3]:
static_df = pd.read_csv("data/owid-covid-data.csv")

In [4]:
static_df

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index
0,AFG,Asia,Afghanistan,2020-01-23,NaN,0.0,NaN,NaN,0.0,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
1,AFG,Asia,Afghanistan,2020-01-24,NaN,0.0,NaN,NaN,0.0,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
2,AFG,Asia,Afghanistan,2020-01-25,NaN,0.0,NaN,NaN,0.0,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
3,AFG,Asia,Afghanistan,2020-01-26,NaN,0.0,NaN,NaN,0.0,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
4,AFG,Asia,Afghanistan,2020-01-27,NaN,0.0,NaN,NaN,0.0,NaN,...,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83,0.498
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62196,ZWE,Africa,Zimbabwe,2020-12-08,10912.0,73.0,111.857,303.0,9.0,3.714,...,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.535
62197,ZWE,Africa,Zimbabwe,2020-12-09,11007.0,95.0,125.429,304.0,1.0,3.857,...,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.535
62198,ZWE,Africa,Zimbabwe,2020-12-10,11081.0,74.0,93.857,305.0,1.0,3.571,...,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.535
62199,ZWE,Africa,Zimbabwe,2020-12-11,11162.0,81.0,87.857,306.0,1.0,3.571,...,1899.775,21.4,307.846,1.82,1.6,30.7,36.791,1.7,61.49,0.535


Our static dataset will have all 180 countries as the rows and density, median age and hdi as the columns in that order. 

In [9]:
dictionary = {'location':[], 'population_density':[], 'median_age':[], 'human_development_index':[]}
count = 0
fields_2 = []
data = []
for i in static_df['location']:
    if i not in dictionary['location']:
            dictionary['location'].append(i)
            dictionary['population_density'].append(round(static_df.loc[count, 'population_density'],3))
            dictionary['median_age'].append(static_df.loc[count, 'median_age'])
            dictionary['human_development_index'].append(round(static_df.loc[count,'human_development_index'],3))
            
    count += 1 
            
for i in dictionary:
    #print(len(dictionary[i]))
    fields_2.append(i)
    data.append(dictionary[i])


        

In [11]:
from itertools import zip_longest
export_data = zip_longest(*data, fillvalue = '')
with open('static.csv', 'w', encoding="ISO-8859-1", newline='') as myfile:
      wr = csv.writer(myfile)
      wr.writerow(fields_2)
      wr.writerows(export_data)
myfile.close()

In [18]:
%cd data

/Users/sbhasin/Desktop/PandemicChallenge/data


Now I will be adding testing and mobility in one dataframe, only for India for now

In [19]:
mob_vars = ['Grocers', 'Parks', 'PublicTransport', 'Residential', 'Workplace']
related = ['date', 'grocery_and_pharmacy', 'parks', 'transit_stations', 'residential', 'workplaces', 'total_tests']
fields_3 = ['date']
for i in related[1:-1]:
    fields_3.append('Mobility_{}'.format(i))
fields_3.append('total_tests')
data = []
dates = []
count = 0
test_data = {}
for i in range(len(mob_vars)+1):      
    temp = []
    k = 0
    writer = csv.writer(myfile)
    if i<5:
        df = pd.read_csv('Mobility_{}.csv'.format(mob_vars[i]))
    else:
        df = pd.read_csv('Testing.csv')
    if count == 0:
        for date in df['Date']:
            if df.loc[count, 'Entity']== 'India':
                dates.append(date)
            count+=1 
        data.append(dates)  
    for country in df['Entity']:
        if country == 'India':
            if i < 5:
                temp.append(round(df.loc[k, related[i+1]],3))
            else:
                if df.loc[k,'Date'] in dates:
                    test_data[df.loc[k,'Date']] = round(df.loc[k, related[6]],3)               
        k+=1 
    data.append(temp)
#print(data)
export_data = zip_longest(*data, fillvalue = '')
with open('dynamic_2.csv', 'w', encoding="ISO-8859-1", newline='') as myfile: 
      wr = csv.writer(myfile)
      wr.writerow(fields_3)
      wr.writerows(export_data)   
myfile.close()                 

In [20]:
df_1 = pd.read_csv("dynamic.csv")
for i in df_1:
    print(i)

Unnamed: 0
date
Mobility_grocery_and_pharmacy
Mobility_parks
Mobility_transit_stations
Mobility_residential
Mobility_workplaces
total_tests


In [21]:
k = 0
for i in df_1['date']:
    df_1.loc[k, 'total_tests'] = test_data.get(i,'nan')
    k+=1 
df_1 = df_1.round(decimals = 3)
df_1 = df_1.drop([df_1.columns[0]] ,  axis='columns')
df_1.to_csv('dynamic_1.csv')

THIRD STAGE -  COMPILING ALL SECONDARY CSV FILES TO ONE LARGE CSV

/Users/sbhasin/Desktop


In [24]:
filenames = ["c1_flag.csv", "c1_school_closing.csv", "c2_flag.csv", "c2_workplace_closing.csv", "c3_cancel_public_events.csv",
            "c4_restrictions_on_gatherings.csv", "c5_close_public_transport.csv", "c6_stay_at_home_requirements.csv", "c7_movementrestrictions.csv",
            "c8_internationaltravel.csv", "c3_flag.csv", "c4_flag.csv", "c5_flag.csv", "c6_flag.csv", "c7_flag.csv" ] 

time = 60
output_date = datetime.date(2020, 11, 30)
print(output_date)
rows = 10
fields_npi = []
for i in filenames:
    fields_npi.append(i[:-4])

fields_dyn = fields_3[1:]
fields_stat = fields_2

2020-11-30


In [120]:
# inserting npis into the dataframe 
final_df = pd.DataFrame()

count = 0
date_dic= {}
for npi_file in filenames:

    India = 0
    count += 1 
    npi_df = pd.read_csv(os.path.join('PandemicChallenge/timeseries', npi_file))
    # first we need to convert the header date formats to more y -m - d format
    headers = []
    for header in npi_df:
        headers.append(header)
    for i in range(3,len(headers)):
        dt = parse(headers[i])
        headers[i] = dt.strftime('%Y-%m-%d')
    npi_df.columns = headers
    #print(npi_df)
    # now we need to find the position of India within the csv files
    for country in npi_df['country_name']:
        if country == 'India':
            break           
        India += 1 
         
    for column in range(1, 1+time):
        new  = pd.Series([], dtype = 'float64')
        current_date = output_date - datetime.timedelta(column)
        for row in range(rows):
            new_date = current_date - datetime.timedelta(row)
            new[row] = npi_df.loc[India, str(new_date)]          
        
        final_df.insert(time*(count-1)+column-1, fields_npi[count-1][:-4] + str(column), new)

index = pd.Series([], dtype = 'float64')
current_dates = []
for row in range(rows):
    new_date = output_date - datetime.timedelta(row)
    index[row] = new_date
    current_dates.append(new_date)
final_df.insert(0, 'current_date', index)
final_df.to_csv(os.path.join('PandemicChallenge/timeseries', 'final.csv'))        


In [121]:
df_1_copy = df_1.copy()
index = []
t = 0
mapper = {}
for i in df_1['date']:
    mapper[i]= t
    index.append(i)
    t+=1 



In [122]:
#now we add our dynamic variables from dynamic.csv
copy = final_df.copy()
copy_2 = final_df.copy()
for header in df_1_copy[2:]:
    if header=='date':
        continue
    else:
        for column in range(1, 1+time):
            new  = pd.Series([], dtype = 'float64')
            current_date = output_date - datetime.timedelta(column)
            date_dic[str(current_date)] = date_dic.get(str(current_date),0) + 1 
            for row in range(rows):
                new_date = current_date - datetime.timedelta(row)
                ind = mapper[str(new_date)]
                new[row] = df_1_copy.loc[mapper[str(new_date)], header]  
            copy.insert(time*(count)+column, header + str(column), new)
    
    

In [123]:
copy

,current_date,c1_1,c1_2,c1_3,c1_4,c1_5,c1_6,c1_7,c1_8,c1_9,...,Mobility_grocery_and_pharmacy51,Mobility_grocery_and_pharmacy52,Mobility_grocery_and_pharmacy53,Mobility_grocery_and_pharmacy54,Mobility_grocery_and_pharmacy55,Mobility_grocery_and_pharmacy56,Mobility_grocery_and_pharmacy57,Mobility_grocery_and_pharmacy58,Mobility_grocery_and_pharmacy59,Mobility_grocery_and_pharmacy60
0,2020-11-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.429,3.286,3.286,2.857,2.143,1.286,0.429,-0.429,-1.286,-3.000
1,2020-11-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.286,3.286,2.857,2.143,1.286,0.429,-0.429,-1.286,-3.000,-4.000
2,2020-11-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.286,2.857,2.143,1.286,0.429,-0.429,-1.286,-3.000,-4.000,-4.857
3,2020-11-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.857,2.143,1.286,0.429,-0.429,-1.286,-3.000,-4.000,-4.857,-5.143
4,2020-11-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.143,1.286,0.429,-0.429,-1.286,-3.000,-4.000,-4.857,-5.143,-5.571
5,2020-11-25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.286,0.429,-0.429,-1.286,-3.000,-4.000,-4.857,-5.143,-5.571,-6.000
6,2020-11-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.429,-0.429,-1.286,-3.000,-4.000,-4.857,-5.143,-5.571,-6.000,-6.286
7,2020-11-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.429,-1.286,-3.000,-4.000,-4.857,-5.143,-5.571,-6.000,-6.286,-6.000
8,2020-11-22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.286,-3.000,-4.000,-4.857,-5.143,-5.571,-6.000,-6.286,-6.000,-6.714
9,2020-11-21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-3.000,-4.000,-4.857,-5.143,-5.571,-6.000,-6.286,-6.000,-6.714,-6.143


In [124]:
# here we add our static variables 
static = (450.419, 28.2, 0.64)
fields = ['population_density' ,'median_age', 'hdi']
for i in range(3):
    #if i!=2:
        #copy = copy.drop(fields[i])
    new  = pd.Series([], dtype = 'float64')
    for j in range(rows):
        new[j] = static[i]
    
    copy.insert(copy.shape[1], fields[i], new)

In [125]:
copy

,current_date,c1_1,c1_2,c1_3,c1_4,c1_5,c1_6,c1_7,c1_8,c1_9,...,Mobility_grocery_and_pharmacy54,Mobility_grocery_and_pharmacy55,Mobility_grocery_and_pharmacy56,Mobility_grocery_and_pharmacy57,Mobility_grocery_and_pharmacy58,Mobility_grocery_and_pharmacy59,Mobility_grocery_and_pharmacy60,population_density,median_age,hdi
0,2020-11-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.857,2.143,1.286,0.429,-0.429,-1.286,-3.000,450.419,28.2,0.64
1,2020-11-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.143,1.286,0.429,-0.429,-1.286,-3.000,-4.000,450.419,28.2,0.64
2,2020-11-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.286,0.429,-0.429,-1.286,-3.000,-4.000,-4.857,450.419,28.2,0.64
3,2020-11-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.429,-0.429,-1.286,-3.000,-4.000,-4.857,-5.143,450.419,28.2,0.64
4,2020-11-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.429,-1.286,-3.000,-4.000,-4.857,-5.143,-5.571,450.419,28.2,0.64
5,2020-11-25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-1.286,-3.000,-4.000,-4.857,-5.143,-5.571,-6.000,450.419,28.2,0.64
6,2020-11-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-3.000,-4.000,-4.857,-5.143,-5.571,-6.000,-6.286,450.419,28.2,0.64
7,2020-11-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-4.000,-4.857,-5.143,-5.571,-6.000,-6.286,-6.000,450.419,28.2,0.64
8,2020-11-22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-4.857,-5.143,-5.571,-6.000,-6.286,-6.000,-6.714,450.419,28.2,0.64
9,2020-11-21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-5.143,-5.571,-6.000,-6.286,-6.000,-6.714,-6.143,450.419,28.2,0.64


In [126]:
# adding the cases growth rates 
output_df = pd.read_csv(os.path.join('PandemicChallenge/timeseries', 'confirmed_cases.csv'))
count = 0
headers = []
for header in output_df:
    headers.append(header)
for i in range(3,len(headers)):
    dt = parse(headers[i])
    headers[i] = dt.strftime('%Y-%m-%d')
output_df.columns = headers
for country in output_df['country_name']:
    if country == "India":
        break
    count +=1 
outputs  = pd.Series([], dtype = 'float64')
for date in range(len(current_dates)):
    prev_date = current_dates[date] - datetime.timedelta(1)
    outputs[date] = (output_df.loc[count, str(current_dates[date])] - output_df.loc[count, str(prev_date)])*100/output_df.loc[count, str(prev_date)]

copy.insert(copy.shape[1],'output', outputs)




In [127]:
# adding the temperatures
temp_df = pd.read_csv(os.path.join('PandemicChallenge/data', 'temperatures.csv'))
month_mapper = {}
count = 0
India = 0
for header in temp_df:
    month_mapper[count] = header
    count+=1 

for country in temp_df['ISO_3DIGIT']:
    if country == "IND":
        break
    India +=1 
prev_date = output_date
for week in range(1,9):
    #copy.drop(columns = ['temp_week_{}'.format(str(week))])
    prev_date = prev_date - datetime.timedelta(7)
    month = int(str(prev_date)[5:-3])
    print(month_mapper[month])
    new = pd.Series([], dtype = 'float64')
    column = month_mapper[month]
    temperature = temp_df.loc[India, column]
    for row in range(rows):
        new[row] = temperature
    copy.insert(copy.shape[1]-1,'temp_week_{}'.format(str(week)), new)


    
    

Nov_Temp
Nov_Temp
Nov_Temp
Nov_Temp
Oct_temp
Oct_temp
Oct_temp
Oct_temp


In [130]:
copy.to_csv(os.path.join('PandemicChallenge/timeseries', 'final.csv'))

In [128]:
copy

,current_date,c1_1,c1_2,c1_3,c1_4,c1_5,c1_6,c1_7,c1_8,c1_9,...,hdi,temp_week_1,temp_week_2,temp_week_3,temp_week_4,temp_week_5,temp_week_6,temp_week_7,temp_week_8,output
0,2020-11-30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.64,20.79,20.79,20.79,20.79,24.52,24.52,24.52,24.52,0.329930
1,2020-11-29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.64,20.79,20.79,20.79,20.79,24.52,24.52,24.52,24.52,0.412779
2,2020-11-28,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.64,20.79,20.79,20.79,20.79,24.52,24.52,24.52,24.52,0.447113
3,2020-11-27,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.64,20.79,20.79,20.79,20.79,24.52,24.52,24.52,24.52,0.443855
4,2020-11-26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.64,20.79,20.79,20.79,20.79,24.52,24.52,24.52,24.52,0.464912
5,2020-11-25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.64,20.79,20.79,20.79,20.79,24.52,24.52,24.52,24.52,0.482411
6,2020-11-24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.64,20.79,20.79,20.79,20.79,24.52,24.52,24.52,24.52,0.483512
7,2020-11-23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.64,20.79,20.79,20.79,20.79,24.52,24.52,24.52,24.52,0.415488
8,2020-11-22,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.64,20.79,20.79,20.79,20.79,24.52,24.52,24.52,24.52,0.484388
9,2020-11-21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.64,20.79,20.79,20.79,20.79,24.52,24.52,24.52,24.52,0.499514


In [80]:
for i in copy:
    print(i)

current_date
c1_1
c1_2
c1_3
c1_4
c1_5
c1_6
c1_7
c1_8
c1_9
c1_10
c1_11
c1_12
c1_13
c1_14
c1_15
c1_16
c1_17
c1_18
c1_19
c1_20
c1_21
c1_22
c1_23
c1_24
c1_25
c1_26
c1_27
c1_28
c1_29
c1_30
c1_31
c1_32
c1_33
c1_34
c1_35
c1_36
c1_37
c1_38
c1_39
c1_40
c1_41
c1_42
c1_43
c1_44
c1_45
c1_46
c1_47
c1_48
c1_49
c1_50
c1_51
c1_52
c1_53
c1_54
c1_55
c1_56
c1_57
c1_58
c1_59
c1_60
c1_school_clo1
c1_school_clo2
c1_school_clo3
c1_school_clo4
c1_school_clo5
c1_school_clo6
c1_school_clo7
c1_school_clo8
c1_school_clo9
c1_school_clo10
c1_school_clo11
c1_school_clo12
c1_school_clo13
c1_school_clo14
c1_school_clo15
c1_school_clo16
c1_school_clo17
c1_school_clo18
c1_school_clo19
c1_school_clo20
c1_school_clo21
c1_school_clo22
c1_school_clo23
c1_school_clo24
c1_school_clo25
c1_school_clo26
c1_school_clo27
c1_school_clo28
c1_school_clo29
c1_school_clo30
c1_school_clo31
c1_school_clo32
c1_school_clo33
c1_school_clo34
c1_school_clo35
c1_school_clo36
c1_school_clo37
c1_school_clo38
c1_school_clo39
c1_school_clo40
c1_sc